In [4]:
from matplotlib import pyplot as plt
from itertools import product
import seaborn as sns
import numpy as np
import pandas as pd
import os.path as op
import argparse
import re
# EEG utilities
# import mne
# from pycrostates.cluster import ModKMeans
# from pycrostates.io import read_cluster, ChData
# # BIDS utilities
# from util.io.bids import DataSink
from bids import BIDSLayout

In [5]:
# constants
BIDS_ROOT = '../data/bids'
DERIV_ROOT = op.join(BIDS_ROOT, 'derivatives')
FFR_PASSBAND = (30., 300.)
TASK = 'pitch'
layout = BIDSLayout(BIDS_ROOT, derivatives = True)

/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/bids/layout/validation.py:151: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


In [6]:
def read_epochs(sub, desc):
    '''
    reads and concatenates epochs across runs
    '''
    layout = BIDSLayout(BIDS_ROOT, derivatives = True)
    run = lambda f: int(re.findall('run-(\w+)_', f)[0])
    fnames = layout.get(
        return_type = 'filename',
        subject = sub, 
        desc = desc
        )
    print(fnames)
    fnames.sort(key = run)
    epochs_all = [mne.read_epochs(f) for f in fnames]
    epochs = mne.concatenate_epochs(epochs_all)
    epochs = epochs.pick('eeg')
    return epochs

def compute_spectrum_dB(epo):
    '''
    Computes power spectrum of frequency following response
    '''
    poststim = epo.average().compute_psd(tmin = 0., tmax = 0.25)
    baseline = epo.average().compute_psd(tmin = -0.25, tmax = 0.)
    power = 10 * np.log10(poststim.get_data() / baseline.get_data())
    power = np.squeeze(power)
    return poststim.freqs, power

In [9]:
# Read behavioral scores
accuracies = pd.read_csv('accuracies.csv')
accuracies

,sub_num,accuracy
0,1,0.952563
1,2,0.901232
2,3,0.785415
3,4,0.958154
4,5,0.919971
5,6,0.921653
6,7,0.911613
7,8,0.989489
8,9,0.912759
9,10,0.965335


In [ ]:
subs = layout.get_subjects(scope = 'preprocessing')
subs.sort(key = int)
all_spectrums = []

for sub in subs:
    
    # read FFR epochs and assign microstate labels
    epochs = read_epochs(sub, 'clean')
    freqs, spectrum = compute_spectrum_dB(epo)